<a href="https://www.kaggle.com/code/dhirajmishra57/udemy-pyspark-tutorial?scriptVersionId=134089138" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Pyspark End-To-End

`Course - https://www.udemy.com/course/pyspark-end-to-end-developer-course-spark-with-python/learn/lecture/37180040#overview`

## Topics -
    - How to read flat file and tables using jbdc connection
    - DataFrame API Selection
    - DataFrame Where & Filter
    - DataFrame API Sorting
    - DataFrame API Set
    - DataFrame API Join
    - DataFrame API Windowing
    - DataFrame API Aggregation
    - DataFrame API GroupBy
    - DataFrame APIs Null Functions

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=1b905cbd4b41bb1b21033930ac8a940fb9d2d9b4e814c7d762a54dc99b98a2d1
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession,functions
from pyspark.sql.functions import *
from pyspark.sql.types import StructField,StructType,IntegerType,StringType
from pyspark.sql import window
from pyspark.sql.window import *

spark = SparkSession.builder.appName("Pyspark-Tutorial").getOrCreate()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 07:32:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/kaggle/input/retail-db-tutorial/RetailDB SalesData/Customers/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Orders/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Products/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Order_items/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Categories/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Departments/part-00000


## How to read file using various techniques

In [3]:
ord_file_path = "/kaggle/input/retail-db-tutorial/RetailDB SalesData/Orders"
ord = spark.read.load(ord_file_path,sep=",",format='csv',schema=('ordid int,ord_date timestamp,ord_cust_id int, ord_status string'))
ord.show(1)

ord = spark.read.csv(ord_file_path,sep=",",schema=('ordid int,ord_date timestamp,ord_cust_id int, ord_status string'))
ord.show(1)

ord = spark.read.format('csv').load(ord_file_path,sep=',',schema='ordid int,ord_date timestamp,ord_cust_id int, ord_status string')
ord.show(1)

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|    1|2013-07-25 00:00:00|      11599|    CLOSED|
+-----+-------------------+-----------+----------+
only showing top 1 row

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|    1|2013-07-25 00:00:00|      11599|    CLOSED|
+-----+-------------------+-----------+----------+
only showing top 1 row

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|    1|2013-07-25 00:00:00|      11599|    CLOSED|
+-----+-------------------+-----------+----------+
only showing top 1 row



## How to read table and jdbc connection

In [4]:
"""
spark.read.jdbc("connection string / jdbc:mysql://localhost:3306/emp","table",
                properties={"user":"root","password":"root","driver":"com.mysql.cj.jdbc.Driver"})

spark.read.format("jdbc") \
.option("url","connection string") \
.option("driver","com.mysql.cj.jdbc.Driver") \
.option("user","root")
.option("password":"root")
.option("dbtable","employee")
# .option("dbtable","select empid, salary from employee where gender = 'M' ")
# .option("query"," select empid, salary from employee where gender = 'M' ")
.load()

"""
print("Hello")

Hello


# DataFrame API Selection

### Select
Select one or more columns
Applying necessary functions on select

### SelectExpr
This is a variant of select that accepts SQL expressions.
If we want to use any functions available in SQL but not in Spark Built-in functions, then we can use selectExpr
            -- explode

### withColumn(colName, col)
Applied transformation to only selected columns.
The first argument is an alias name. If we give an alias name same as a column name, the transformations will apply
on the same column.
Otherwise a new column will be formed. Avoid giving alias name same as column name.
ord.withColumn('order_month',substring(orderDF.order_date,1,10)).show()


### withColumnRenamed(existingCol, newCol)
Rename Existing Column.
ord.withColumnRenamed('order_id','order_id1').show()

### drop(*cols)
Drop a column.
order = ord.drop('order_id','order_date')

### dropDuplicates(subset=None)
Drop duplicate rows.
Optionally can consider only subset of columns.
emp.dropDuplicates().show()
emp.dropDuplicates(("name","score1","score2")).show()

In [5]:
ord
data = [('Robert',35,40,40),('Robert',35,40,40),('Ram',31,33,29),('Ram',31,33,91)]
emp = spark.createDataFrame(data=data,schema=['name','score1','score2','score3'])

In [6]:
ord.select("ordid",'ordid',ord.ordid).show(5)

ord.select(lower("ord_status").alias("lower_sts"),"ord_status").show(5)

+-----+-----+-----+
|ordid|ordid|ordid|
+-----+-----+-----+
|    1|    1|    1|
|    2|    2|    2|
|    3|    3|    3|
|    4|    4|    4|
|    5|    5|    5|
+-----+-----+-----+
only showing top 5 rows

+---------------+---------------+
|      lower_sts|     ord_status|
+---------------+---------------+
|         closed|         CLOSED|
|pending_payment|PENDING_PAYMENT|
|       complete|       COMPLETE|
|         closed|         CLOSED|
|       complete|       COMPLETE|
+---------------+---------------+
only showing top 5 rows



In [7]:
# ord.show(5)
ord.selectExpr("substring(ord_date,1,4) as ord_year","ord_date") \
.selectExpr("ord_year","substring(ord_date,6,2) as ord_month","ord_date") \
.selectExpr("ord_date","ord_month","ord_year","substring(ord_date,9,2) as ord_day") \
.select("ord_year","ord_month","ord_date") \
.show(5)

+--------+---------+-------------------+
|ord_year|ord_month|           ord_date|
+--------+---------+-------------------+
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
+--------+---------+-------------------+
only showing top 5 rows



In [8]:
ord.withColumn("ord_year",substring("ord_date",1,4)).show(5)

ord.withColumn("ord_date",substring("ord_date",1,4)).show(5)

+-----+-------------------+-----------+---------------+--------+
|ordid|           ord_date|ord_cust_id|     ord_status|ord_year|
+-----+-------------------+-----------+---------------+--------+
|    1|2013-07-25 00:00:00|      11599|         CLOSED|    2013|
|    2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|    2013|
|    3|2013-07-25 00:00:00|      12111|       COMPLETE|    2013|
|    4|2013-07-25 00:00:00|       8827|         CLOSED|    2013|
|    5|2013-07-25 00:00:00|      11318|       COMPLETE|    2013|
+-----+-------------------+-----------+---------------+--------+
only showing top 5 rows

+-----+--------+-----------+---------------+
|ordid|ord_date|ord_cust_id|     ord_status|
+-----+--------+-----------+---------------+
|    1|    2013|      11599|         CLOSED|
|    2|    2013|        256|PENDING_PAYMENT|
|    3|    2013|      12111|       COMPLETE|
|    4|    2013|       8827|         CLOSED|
|    5|    2013|      11318|       COMPLETE|
+-----+--------+-----------+--

In [9]:
ord.withColumnRenamed("ord_status","ord_sts").show(5)

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|        ord_sts|
+-----+-------------------+-----------+---------------+
|    1|2013-07-25 00:00:00|      11599|         CLOSED|
|    2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|    3|2013-07-25 00:00:00|      12111|       COMPLETE|
|    4|2013-07-25 00:00:00|       8827|         CLOSED|
|    5|2013-07-25 00:00:00|      11318|       COMPLETE|
+-----+-------------------+-----------+---------------+
only showing top 5 rows



In [10]:
ord.drop('ord_status','ord_date').show(2)
ord.drop('ord_status').show(2)

+-----+-----------+
|ordid|ord_cust_id|
+-----+-----------+
|    1|      11599|
|    2|        256|
+-----+-----------+
only showing top 2 rows

+-----+-------------------+-----------+
|ordid|           ord_date|ord_cust_id|
+-----+-------------------+-----------+
|    1|2013-07-25 00:00:00|      11599|
|    2|2013-07-25 00:00:00|        256|
+-----+-------------------+-----------+
only showing top 2 rows



In [11]:
emp.show()

emp.dropDuplicates().show() ## only 1 record filtered

emp.dropDuplicates(["name"]).show()

emp.dropDuplicates(["name","score2"]).show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    91|
+------+------+------+------+



+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    91|
+------+------+------+------+



+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
+------+------+------+------+



+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
+------+------+------+------+



# DataFrame API Filter

### filter(condition): (Its alias ‘where’)
Filter rows using a given condition

use '&' for 'and‘. '|' for 'or‘. (boolean expressions)

Use column function isin() for multiple search Or use IN Operator for SQL Style syntax.

In [12]:
# &
ord.where((ord.ordid > 10) & (ord.ordid < 20)).show(5)

ord.filter((ord.ordid > 10) & (ord.ordid < 20)).show(5)

# |
ord.where((ord.ordid == 10) | (ord.ordid == 20)).show(5)

ord.where((ord.ordid == 10) | (ord.ordid == 20)).show(5)

# isin
ord.where(ord.ord_status.isin("CLOSED","PENDING")).show(5)
ord.filter(ord.ord_status.isin("CLOSED","PENDING")).show(5)

# sql like where optional
ord.where("ord_status in ('CLOSED','NOT-PENDING') ").show(5)

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|   12|2013-07-25 00:00:00|       1837|         CLOSED|
|   13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|   14|2013-07-25 00:00:00|       9842|     PROCESSING|
|   15|2013-07-25 00:00:00|       2568|       COMPLETE|
+-----+-------------------+-----------+---------------+
only showing top 5 rows

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|   12|2013-07-25 00:00:00|       1837|         CLOSED|
|   13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|   14|2013-07-25 00:00:00|       9842|     PROCESSING|
|   15|2013-07-25 00:00:00|       2568|       COMPLETE|
+-----+----------------

# DataFrame APIs Sort


### sort() or orderBy()
orderBy() is an alias for .sort()
Sort specific column(s).

### sortWithinPartitions
At time, we may not want sort globally, but with in a group. In that case we can use sortWithinPartitions.

In [13]:
ord.sort(ord.ord_status.desc()).show(2)
ord.sort(col("ord_status").desc()).show(2)
ord.sort("ord_status","ord_cust_id").show(2)
ord.sort("ord_status","ord_cust_id",ascending=[0,1]).show(2)

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   69|2013-07-25 00:00:00|       2821|SUSPECTED_FRAUD|
|  246|2013-07-26 00:00:00|       9616|SUSPECTED_FRAUD|
+-----+-------------------+-----------+---------------+
only showing top 2 rows

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   69|2013-07-25 00:00:00|       2821|SUSPECTED_FRAUD|
|  246|2013-07-26 00:00:00|       9616|SUSPECTED_FRAUD|
+-----+-------------------+-----------+---------------+
only showing top 2 rows

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|19709|2013-11-24 00:00:00|         18|  CANCELED|
|11256|2013-10-02 00:00:00|         35|  CANCELED|
+-----+----------------

In [14]:
data = [("a",1),("b",2),("c",3),("d",4),("e",5)]
df = spark.createDataFrame(data,schema='col1 string,col2 int')
df = df.repartition(2)
df = df.withColumn("partid",spark_partition_id())

df.sort(df.col1).show()
df.sortWithinPartitions(df.col1).show()

+----+----+------+
|col1|col2|partid|
+----+----+------+
|   a|   1|     1|
|   b|   2|     1|
|   c|   3|     0|
|   d|   4|     0|
|   e|   5|     1|
+----+----+------+

+----+----+------+
|col1|col2|partid|
+----+----+------+
|   c|   3|     0|
|   d|   4|     0|
|   a|   1|     1|
|   b|   2|     1|
|   e|   5|     1|
+----+----+------+



## DataFrame APIs : Set Operators

### union() and unionAll():
Same and contains duplicate values
Use distinct after union or unionAll to remove duplicates

### unionByName():
The difference between this function and :func:`union` is that this function
resolves columns by name (not by position)

### intersect(): 
Containing rows in both DataFrames. Removed duplicates

### intersectAll():
Same as intersect. But retains the duplicates

### exceptAll():
Rows present in one DataFrame but not in another

In [15]:
df1 = spark.range(4)
df2 = spark.range(3,5)

# Union and UnionAll performs same way

df1.union(df2).show()
df1.unionAll(df2).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  3|
|  4|
+---+

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  3|
|  4|
+---+



In [16]:
df1.intersect(df2).show()
df1.intersectAll(df2).show()

+---+
| id|
+---+
|  3|
+---+

+---+
| id|
+---+
|  3|
+---+



# DataFrame APIs : Join

Join Type
inner INNER JOIN, JOIN
outer, full, fullouter,
full_outer
FULL OUTER JOIN
left,
left_outer,leftouter
LEFT JOIN
right,right_outer,rig
htouter
RIGHT JOIN
cross CROSS JOIN
left_anti, leftanti
leftsemi,left_semi

In [17]:
df1 = spark.createDataFrame(data=[(1,'Robert'),(2,'Ria'),(3,'James')],schema='empid int, empname string')
df2 = spark.createDataFrame(data=[(2,'USA'),(4,'India')],schema='empid int, country string')

# Inner Join
df1.join(df2,df1.empid == df2.empid).select(df1.empid.alias("employeeid"),df1.empname,df2.country).show()

# left anti join
# cannot get right table attributes and it executes faster as compared to inner as it doesn't check right/second table attributes
df1.join(df2,df1.empid == df2.empid,"leftanti").show()

# Left outer or Left
df1.join(df2,df1.empid == df2.empid,"left")\
.select(df1.empid,df1.empname,df2.country) \
.sort(df2.country.desc()) \
.show()

# full outer
df1.join(df2,df1.empid == df2.empid,"full")\
.select(df1.empid,df1.empname,df2.country) \
.sort(df1.empid.asc()) \
.show()

+----------+-------+-------+
|employeeid|empname|country|
+----------+-------+-------+
|         2|    Ria|    USA|
+----------+-------+-------+



+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    3|  James|
+-----+-------+



+-----+-------+-------+
|empid|empname|country|
+-----+-------+-------+
|    2|    Ria|    USA|
|    1| Robert|   null|
|    3|  James|   null|
+-----+-------+-------+



+-----+-------+-------+
|empid|empname|country|
+-----+-------+-------+
| null|   null|  India|
|    1| Robert|   null|
|    2|    Ria|    USA|
|    3|  James|   null|
+-----+-------+-------+



In [18]:
# self

df1 = spark.createDataFrame(data=((1,'Robert',2),(2,'Ria',3),(3,'James',5)),schema='empid int,empname string,managerid int')
df1.show()
df1.alias("emp1").join(df1.alias("emp2"),col("emp1.managerid") == col("emp2.empid"))\
.select(col("emp1.empid").alias("EmployeeID"),col("emp1.empname").alias("EmployeeName"),col("emp2.empid").alias("ManagerID"),col("emp2.empname").alias("ManagerName")) \
.show()

+-----+-------+---------+
|empid|empname|managerid|
+-----+-------+---------+
|    1| Robert|        2|
|    2|    Ria|        3|
|    3|  James|        5|
+-----+-------+---------+



+----------+------------+---------+-----------+
|EmployeeID|EmployeeName|ManagerID|ManagerName|
+----------+------------+---------+-----------+
|         1|      Robert|        2|        Ria|
|         2|         Ria|        3|      James|
+----------+------------+---------+-----------+



### Window Functions

Lets see 3 types of Window Functions:
Ranking
Analytical
Aggregate

In [19]:
data = (("James","Sales","NY",9000,34),
("Alicia","Sales","NY",8600,56),
("Robert","Sales","CA",8100,30),
("John","Sales","AZ",8600,31),
("Ross","Sales","AZ",8100,33),
("Kathy","Sales","AZ",1000,39),
("Lisa","Finance","CA",9000,24),
("Deja","Finance","CA",9900,40),
("Sugie","Finance","NY",8300,36),
("Ram","Finance","NY",7900,53),
("Satya","Finance","AZ",8200,53),
("Kyle","Marketing","CA",8000,25),
("Reid","Marketing","NY",9100,50)
)

schema=("empname","dept","state","salary","age")

df = spark.createDataFrame(data=data,schema=schema)

In [20]:
spec = Window.partitionBy("dept").orderBy(col("salary").desc())

df.withColumn("rank",rank().over(spec))\
.withColumn("drank",dense_rank().over(spec))\
.withColumn("rownum",row_number().over(spec))\
.withColumn("perrnk",percent_rank().over(spec))\
.withColumn("perrnk",percent_rank().over(spec))\
.show()

+-------+---------+-----+------+---+----+-----+------+------+
|empname|     dept|state|salary|age|rank|drank|rownum|perrnk|
+-------+---------+-----+------+---+----+-----+------+------+
|   Deja|  Finance|   CA|  9900| 40|   1|    1|     1|   0.0|
|   Lisa|  Finance|   CA|  9000| 24|   2|    2|     2|  0.25|
|  Sugie|  Finance|   NY|  8300| 36|   3|    3|     3|   0.5|
|  Satya|  Finance|   AZ|  8200| 53|   4|    4|     4|  0.75|
|    Ram|  Finance|   NY|  7900| 53|   5|    5|     5|   1.0|
|   Reid|Marketing|   NY|  9100| 50|   1|    1|     1|   0.0|
|   Kyle|Marketing|   CA|  8000| 25|   2|    2|     2|   1.0|
|  James|    Sales|   NY|  9000| 34|   1|    1|     1|   0.0|
| Alicia|    Sales|   NY|  8600| 56|   2|    2|     2|   0.2|
|   John|    Sales|   AZ|  8600| 31|   2|    2|     3|   0.2|
| Robert|    Sales|   CA|  8100| 30|   4|    3|     4|   0.6|
|   Ross|    Sales|   AZ|  8100| 33|   4|    3|     5|   0.6|
|  Kathy|    Sales|   AZ|  1000| 39|   6|    4|     6|   1.0|
+-------

In [21]:
spec = Window.partitionBy("dept").orderBy(col("salary").desc())

df.select("dept","salary")\
.withColumn("lead_salary",lead("salary",1,-1).over(spec))\
.withColumn("lag_salary",lag("salary",1,-1).over(spec))\
.show()

+---------+------+-----------+----------+
|     dept|salary|lead_salary|lag_salary|
+---------+------+-----------+----------+
|  Finance|  9900|       9000|        -1|
|  Finance|  9000|       8300|      9900|
|  Finance|  8300|       8200|      9000|
|  Finance|  8200|       7900|      8300|
|  Finance|  7900|         -1|      8200|
|Marketing|  9100|       8000|        -1|
|Marketing|  8000|         -1|      9100|
|    Sales|  9000|       8600|        -1|
|    Sales|  8600|       8600|      9000|
|    Sales|  8600|       8100|      8600|
|    Sales|  8100|       8100|      8600|
|    Sales|  8100|       1000|      8100|
|    Sales|  1000|         -1|      8100|
+---------+------+-----------+----------+



In [22]:
spec_without_orderby = Window.partitionBy("dept")
spec_with_orderby = Window.partitionBy("dept").orderBy(col("salary").asc()) ## rolling sum if we use order by

df.select("dept","salary")\
.withColumn("sum_slry_without_orderby",sum("salary").over(spec_without_orderby))\
.withColumn("sum_slry_with_orderby",sum("salary").over(spec_with_orderby))\
.withColumn("avg_slry_without_orderby",avg("salary").over(spec_without_orderby))\
.withColumn("avg_slry_with_orderby",avg("salary").over(spec_with_orderby))\
.withColumn("min_salary",min("salary").over(spec_without_orderby))\
.withColumn("max_salary",max("salary").over(spec_without_orderby))\
.withColumn("count_salary",count("salary").over(spec_without_orderby))\
.show()

+---------+------+------------------------+---------------------+------------------------+---------------------+----------+----------+------------+
|     dept|salary|sum_slry_without_orderby|sum_slry_with_orderby|avg_slry_without_orderby|avg_slry_with_orderby|min_salary|max_salary|count_salary|
+---------+------+------------------------+---------------------+------------------------+---------------------+----------+----------+------------+
|  Finance|  7900|                   43300|                 7900|                  8660.0|               7900.0|      7900|      9900|           5|
|  Finance|  8200|                   43300|                16100|                  8660.0|               8050.0|      7900|      9900|           5|
|  Finance|  8300|                   43300|                24400|                  8660.0|    8133.333333333333|      7900|      9900|           5|
|  Finance|  9000|                   43300|                33400|                  8660.0|               8350.0|

### Window Functions

* rangeBetween:
    - Takes two argument (start,end) to define frame boundaries.
    - Default : unboundedPreceding and unboundedFollowing

* rowsBetween:
    - Takes two argument (start,end) to define frame boundaries.
    - Deafult : unboundedPreceding and unboundedFollowing.

In [23]:
spec1=Window.partitionBy(df.dept).orderBy(df.salary).rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df.select(df.dept,df.salary).withColumn("sum_sal",sum("salary").over(spec1)).show()
spec1=Window.partitionBy(df.dept).orderBy(df.salary).rangeBetween(Window.currentRow, Window.unboundedFollowing)
df.select(df.dept,df.salary).withColumn("sum_sal",sum("salary").over(spec1)).show()
spec1=Window.partitionBy(df.dept).orderBy(df.salary).rangeBetween(Window.currentRow,500)
df.select(df.dept,df.salary).withColumn("sum_sal",sum("salary").over(spec1)).show()

+---------+------+-------+
|     dept|salary|sum_sal|
+---------+------+-------+
|  Finance|  7900|  43300|
|  Finance|  8200|  43300|
|  Finance|  8300|  43300|
|  Finance|  9000|  43300|
|  Finance|  9900|  43300|
|Marketing|  8000|  17100|
|Marketing|  9100|  17100|
|    Sales|  1000|  43400|
|    Sales|  8100|  43400|
|    Sales|  8100|  43400|
|    Sales|  8600|  43400|
|    Sales|  8600|  43400|
|    Sales|  9000|  43400|
+---------+------+-------+

+---------+------+-------+
|     dept|salary|sum_sal|
+---------+------+-------+
|  Finance|  7900|  43300|
|  Finance|  8200|  35400|
|  Finance|  8300|  27200|
|  Finance|  9000|  18900|
|  Finance|  9900|   9900|
|Marketing|  8000|  17100|
|Marketing|  9100|   9100|
|    Sales|  1000|  43400|
|    Sales|  8100|  42400|
|    Sales|  8100|  42400|
|    Sales|  8600|  26200|
|    Sales|  8600|  26200|
|    Sales|  9000|   9000|
+---------+------+-------+

+---------+------+-------+
|     dept|salary|sum_sal|
+---------+------+-------+

# DataFrame APIs : Aggregations

    - Summary
    - avg, min, max, sum, sumDistinct, count, countDistinct -- all can be used in same/single `select`
    - df = spark.createDataFrame(((1,100),(2,150),(3,200),(4,50),(5,50)),schema='id int,salary int')
    - df.select(collect_list(df.salary)).show(truncate=False)


In [24]:
ord_items = "/kaggle/input/retail-db-tutorial/RetailDB SalesData/Order_items/part-00000"

orditems = spark.read.format("csv").load(ord_items,sep=",",schema='order_item_id int,ord_item_order_id int, order_item_product_id int, quantity int,subtotal float, price float')
orditems.summary().show()

23/06/19 07:32:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+---------------------+------------------+------------------+------------------+
|summary|    order_item_id| ord_item_order_id|order_item_product_id|          quantity|          subtotal|             price|
+-------+-----------------+------------------+---------------------+------------------+------------------+------------------+
|  count|           172198|            172198|               172198|            172198|            172198|            172198|
|   mean|          86099.5| 34442.56682423721|    660.4877176273824|2.1821275508426345|199.32066922046081|133.75906959048717|
| stddev|49709.42516431533|19883.325171992343|     310.514472790008|1.4663523175387134|112.74303987146804|118.55893633258484|
|    min|                1|                 1|                   19|                 1|              9.99|              9.99|
|    25%|            43033|             17204|                  403|                 1|            119.98|            

In [25]:
orditems.select("price").summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            172198|
|   mean|133.75906959048717|
| stddev|118.55893633258484|
|    min|              9.99|
|    25%|              50.0|
|    50%|             59.99|
|    75%|            199.99|
|    max|           1999.99|
+-------+------------------+



In [26]:
orditems.select(avg("price").alias("avg_price"),min("price").alias("min_price"),max("price").alias("max_price")).show()

+------------------+---------+---------+
|         avg_price|min_price|max_price|
+------------------+---------+---------+
|133.75906959048717|     9.99|  1999.99|
+------------------+---------+---------+



In [27]:
orditems.select(avg("price").alias("avg_price"),
                min("price").alias("min_price"),
                max("price").alias("max_price"),
                sum("price").alias("sum_price"),
                sumDistinct("price").alias("sum_distinct_price")
               ).show()

/opt/conda/lib/python3.10/site-packages/pyspark/sql/functions.py:752: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+------------------+---------+---------+--------------------+------------------+
|         avg_price|min_price|max_price|           sum_price|sum_distinct_price|
+------------------+---------+---------+--------------------+------------------+
|133.75906959048717|     9.99|  1999.99|2.3033044265342712E7|  9832.41999053955|
+------------------+---------+---------+--------------------+------------------+



In [28]:
orditems.select(avg("price").alias("avg_price"),
                min("price").alias("min_price"),
                max("price").alias("max_price"),
                sum("price").alias("sum_price"),
                sumDistinct("price").alias("sum_distinct_price"),
                count("price").alias("count_price"),
                countDistinct("price").alias("distinct_price_count")
               ).show()

+------------------+---------+---------+--------------------+------------------+-----------+--------------------+
|         avg_price|min_price|max_price|           sum_price|sum_distinct_price|count_price|distinct_price_count|
+------------------+---------+---------+--------------------+------------------+-----------+--------------------+
|133.75906959048717|     9.99|  1999.99|2.3033044265342712E7|  9832.41999053955|     172198|                  57|
+------------------+---------+---------+--------------------+------------------+-----------+--------------------+



In [29]:
df = spark.createDataFrame(((1,100),(2,150),(3,200),(4,50),(5,50)),schema='id int,salary int')
df.show()
df.select(collect_list(df.salary)).show(truncate=False) ## shows duplicate value as well
df.select(collect_set(df.salary)).show(truncate=False)  ## Only shows unique values

+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   150|
|  3|   200|
|  4|    50|
|  5|    50|
+---+------+

+-----------------------+
|collect_list(salary)   |
+-----------------------+
|[100, 150, 200, 50, 50]|
+-----------------------+

+-------------------+
|collect_set(salary)|
+-------------------+
|[150, 100, 50, 200]|
+-------------------+



# DataFrame API GroupBy

In [30]:
data = (("James","Sales","NY",9000,34),
("Alicia","Sales","NY",8600,56),
("Robert","Sales","CA",8100,30),
("Lisa","Finance","CA",9000,24),
("Deja","Finance","CA",9900,40),
("Sugie","Finance","NY",8300,36),
("Ram","Finance","NY",7900,53),
("Kyle","Marketing","CA",8000,25),
("Reid","Marketing","NY",9100,50))

schema=("empname","dept","state","salary","age")
df = spark.createDataFrame(data=data,schema=schema)
df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



### Filtering records before performing aggregations

In [31]:
groupped_data = df.where(df.state == "NY").groupBy("dept")
groupped_data.agg(min("salary").alias("min_salary"),max("salary").alias("max_salary"),avg("salary").alias("avg_salary")).show()

+---------+----------+----------+----------+
|     dept|min_salary|max_salary|avg_salary|
+---------+----------+----------+----------+
|    Sales|      8600|      9000|    8800.0|
|  Finance|      7900|      8300|    8100.0|
|Marketing|      9100|      9100|    9100.0|
+---------+----------+----------+----------+



### Filtering records after performing aggregations

In [32]:
groupped_data = df.groupBy("dept")
groupped_data.agg(min("salary").alias("min_salary"),
                  max("salary").alias("max_salary"),
                  avg("salary").alias("avg_salary"))\
.where(col("min_salary") >= 8000)\
.show()

+---------+----------+----------+-----------------+
|     dept|min_salary|max_salary|       avg_salary|
+---------+----------+----------+-----------------+
|    Sales|      8100|      9000|8566.666666666666|
|Marketing|      8000|      9100|           8550.0|
+---------+----------+----------+-----------------+



### Without Filters

In [33]:
groupped_data = df.groupBy("dept")
groupped_data.agg(min("salary").alias("min_salary"),
                  max("salary").alias("max_salary"),
                  avg("salary").alias("avg_salary"))\
.show()

+---------+----------+----------+-----------------+
|     dept|min_salary|max_salary|       avg_salary|
+---------+----------+----------+-----------------+
|    Sales|      8100|      9000|8566.666666666666|
|  Finance|      7900|      9900|           8775.0|
|Marketing|      8000|      9100|           8550.0|
+---------+----------+----------+-----------------+



# DataFrame APIs : Null Functions

In [34]:
# Prepare Data
df = spark.createDataFrame((('Robert',1, None,114.0), ('John',None, 2577,float('nan'))), ("name", "id","phone","stAdd"))
df.show()

+------+----+-----+-----+
|  name|  id|phone|stAdd|
+------+----+-----+-----+
|Robert|   1| null|114.0|
|  John|null| 2577|  NaN|
+------+----+-----+-----+



In [35]:
# To check Null values
df.select("phone",isnull("phone").alias("nullcheck")).show()

## Advance filtering based on true false
df.select("phone",isnull("phone").alias("nullcheck")).filter(col("nullcheck")).show()

+-----+---------+
|phone|nullcheck|
+-----+---------+
| null|     true|
| 2577|    false|
+-----+---------+

+-----+---------+
|phone|nullcheck|
+-----+---------+
| null|     true|
+-----+---------+



In [36]:
# To check NaN == Not a Number values
df.select("stAdd",isnan("stAdd").alias("nullcheck")).show()

## Advance filtering based on true false
df.select("stAdd",isnan("stAdd").alias("nullcheck")).filter(col("nullcheck")).show()

+-----+---------+
|stAdd|nullcheck|
+-----+---------+
|114.0|    false|
|  NaN|     true|
+-----+---------+

+-----+---------+
|stAdd|nullcheck|
+-----+---------+
|  NaN|     true|
+-----+---------+



In [37]:
# Returns col1 if it is not NaN, or col2 if col1 is NaN.
df.select(df.stAdd,df.phone,nanvl(df.stAdd,df.phone)).show()

+-----+-----+-------------------+
|stAdd|phone|nanvl(stAdd, phone)|
+-----+-----+-------------------+
|114.0| null|              114.0|
|  NaN| 2577|             2577.0|
+-----+-----+-------------------+



## Very Important

##### Drop can drop any rows and columns

In [38]:
## Drop can drop columns and rows
data=(('Alice',80,10),('Bob',None,5),('Tom',50,50),(None,None,None),('Robert',30,35))
schema='name string, age int, height int'
df = spark.createDataFrame(data,schema)
df.show()
df.na.drop().show()
df.where(~isnull("age")).drop().show()
df.drop(df.age).show()

+------+----+------+
|  name| age|height|
+------+----+------+
| Alice|  80|    10|
|   Bob|null|     5|
|   Tom|  50|    50|
|  null|null|  null|
|Robert|  30|    35|
+------+----+------+

+------+---+------+
|  name|age|height|
+------+---+------+
| Alice| 80|    10|
|   Tom| 50|    50|
|Robert| 30|    35|
+------+---+------+

+------+---+------+
|  name|age|height|
+------+---+------+
| Alice| 80|    10|
|   Tom| 50|    50|
|Robert| 30|    35|
+------+---+------+

+------+------+
|  name|height|
+------+------+
| Alice|    10|
|   Bob|     5|
|   Tom|    50|
|  null|  null|
|Robert|    35|
+------+------+



In [39]:
df.na.fill(-1).show()

+------+---+------+
|  name|age|height|
+------+---+------+
| Alice| 80|    10|
|   Bob| -1|     5|
|   Tom| 50|    50|
|  null| -1|    -1|
|Robert| 30|    35|
+------+---+------+



In [40]:
### emp2 DataFrame
data=(('Robert',35,40,40),('Ram',31,33,29),('John',95,89,91))
schema = ('name','score1','score2','score3')
emp2= spark.createDataFrame(data=data, schema=schema)

emp2.select(emp2.score1, emp2.score2,sequence(emp2.score1,emp2.score2).alias('new_col')).show(truncate=False)

+------+------+----------------------------+
|score1|score2|new_col                     |
+------+------+----------------------------+
|35    |40    |[35, 36, 37, 38, 39, 40]    |
|31    |33    |[31, 32, 33]                |
|95    |89    |[95, 94, 93, 92, 91, 90, 89]|
+------+------+----------------------------+

